In [2]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection
c:/Users/Admin/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [3]:
save_dir=f"{exps_dir}/exp3/exp_base"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [4]:
test_size=0.33
seed=42
train=pd.read_csv(f'{exps_dir}/exp3/train_EDA.csv')
test=pd.read_csv(f'{exps_dir}/exp3/test_EDA.csv')
train.head()

,isFraud,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,...,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3
0,0,68.5,W,13926.0,NaN,150.0,142.0,credit,315.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,150.0,102.0,credit,325.0,NaN,...,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN,NaN,NaN
2,0,59.0,W,4663.0,490.0,150.0,166.0,debit,330.0,287.0,...,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN,NaN,NaN
3,0,50.0,W,18132.0,567.0,150.0,117.0,debit,476.0,NaN,...,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN,NaN,NaN
4,0,50.0,H,4497.0,514.0,150.0,102.0,credit,420.0,NaN,...,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN,NaN,NaN


In [5]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

True

In [6]:
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [7]:
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [8]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
print(cols_to_drop)
cols_to_drop.remove('isFraud')
len(cols_to_drop)

['card3', 'c7', 'P_emaildomain_3', 'ENTITY_TYPE', 'R_emaildomain_3', 'isFraud']


5

In [9]:
train.shape, test.shape

((561013, 98), (29527, 99))

In [10]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
train.head()

,isFraud,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,...,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3
0,0,68.5,W,13926.0,NaN,150.0,142.0,credit,315.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,150.0,102.0,credit,325.0,NaN,...,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN,NaN,NaN
2,0,59.0,W,4663.0,490.0,150.0,166.0,debit,330.0,287.0,...,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN,NaN,NaN
3,0,50.0,W,18132.0,567.0,150.0,117.0,debit,476.0,NaN,...,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN,NaN,NaN
4,0,50.0,H,4497.0,514.0,150.0,102.0,credit,420.0,NaN,...,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN,NaN,NaN


In [12]:
train.isnull().sum()

isFraud                 0
transactionamt          0
productcd               0
card1                   0
card2                8604
                    ...  
P_emaildomain_2     89737
P_emaildomain_3    558111
R_emaildomain_1    429350
R_emaildomain_2    429443
R_emaildomain_3    558436
Length: 98, dtype: int64

In [13]:
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [14]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns
categorical_cols

Index(['productcd', 'card6', 'p_emaildomain', 'r_emaildomain', 'devicetype',
       'deviceinfo', 'EVENT_ID', 'ENTITY_ID', 'EVENT_TIMESTAMP',
       'P_emaildomain_1', 'P_emaildomain_2', 'R_emaildomain_1',
       'R_emaildomain_2'],
      dtype='object')

In [15]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns

for col in continuous_cols:
    train[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    train[col].fillna(train[col].mode()[0], inplace=True)

train.isnull().sum()

isFraud               0
transactionamt        0
productcd             0
card1                 0
card2                 1
                     ..
id_02_to_std_card6    4
P_emaildomain_1       0
P_emaildomain_2       0
R_emaildomain_1       0
R_emaildomain_2       0
Length: 93, dtype: int64

In [16]:
train = train.drop(['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID'], axis=1)
test = test.drop(['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID'], axis=1)

In [17]:
train=train.dropna()

In [18]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'devicetype', 'deviceinfo', 'productcd', 'card4', 'card6', 'M4','p_emaildomain',
            'r_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))
    else:
        print(f"Column {col} not found in train/test data. Skipping Label Encoding for this column.")   

Column id_12 not found in train/test data. Skipping Label Encoding for this column.
Column id_14 not found in train/test data. Skipping Label Encoding for this column.
Column id_15 not found in train/test data. Skipping Label Encoding for this column.
Column id_16 not found in train/test data. Skipping Label Encoding for this column.
Column id_18 not found in train/test data. Skipping Label Encoding for this column.
Column id_21 not found in train/test data. Skipping Label Encoding for this column.
Column id_22 not found in train/test data. Skipping Label Encoding for this column.
Column id_23 not found in train/test data. Skipping Label Encoding for this column.
Column id_24 not found in train/test data. Skipping Label Encoding for this column.
Column id_25 not found in train/test data. Skipping Label Encoding for this column.
Column id_26 not found in train/test data. Skipping Label Encoding for this column.
Column id_27 not found in train/test data. Skipping Label Encoding for this 

In [19]:
train.columns

Index(['isFraud', 'transactionamt', 'productcd', 'card1', 'card2', 'card5',
       'card6', 'addr1', 'dist1', 'p_emaildomain', 'r_emaildomain', 'c1', 'c2',
       'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'v62',
       'v70', 'v76', 'v78', 'v82', 'v91', 'v127', 'v130', 'v139', 'v160',
       'v165', 'v187', 'v203', 'v207', 'v209', 'v210', 'v221', 'v234', 'v257',
       'v258', 'v261', 'v264', 'v266', 'v267', 'v271', 'v274', 'v277', 'v283',
       'v285', 'v289', 'v291', 'v294', 'id_01', 'id_02', 'id_05', 'id_06',
       'id_09', 'id_13', 'id_17', 'id_19', 'id_20', 'devicetype', 'deviceinfo',
       'transactionamt_to_mean_card1', 'transactionamt_to_mean_card2',
       'transactionamt_to_std_card1', 'transactionamt_to_mean_card3',
       'transactionamt_to_std_card3', 'transactionamt_to_mean_card5',
       'transactionamt_to_std_card5', 'transactionamt_to_mean_card6',
       'transactionamt_to_std_card6', 'transactionamt_to_std_card2',
       'id_02_to_mean_card1'

In [20]:
X_train = train.drop(['isFraud'], axis=1)
y_train = train['isFraud']


# del train
# test = test[["TransactionDT", 'TransactionID']]

In [21]:
from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, random_state = 101)
X_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)
# x_valid.to_csv(f'{save_dir}/x_valid.csv', index=False)
# y_valid.to_csv(f'{save_dir}/y_valid.csv', index=False)

In [22]:
y_train.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [23]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.5179997008359972, 1: 14.38911972914743}

In [24]:
# from imblearn.over_sampling import SMOTE
# x_train, y_train = SMOTE().fit_resample(x_train, y_train)


In [25]:
test = test.drop(['EVENT_ID.1'], axis=1)
for col in continuous_cols:
    if col in ['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID']:
        continue
    test[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    if col in ['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID']:
        continue
    test[col].fillna(test[col].mode()[0], inplace=True)
test=test.dropna()
x_test = test.drop(['isFraud'], axis=1)
y_test = test['isFraud']

x_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)